In [1]:
#hide

In [4]:
#hide
import utils
utils.hero("Using librosa's tempo detector")

Usign `librosa` tempo detector is straight forward.

In [5]:
import librosa

In [4]:
tempo_comparison_report_df = pd.read_csv("../../../meluron-toolbox/musa/data/reports/20250623-151215_bpm_report.csv")
tempo_comparison_report_summary = pd.read_csv("../../../meluron-toolbox/musa/data/reports/20250623-151215_bpm_summary.csv")

In [5]:
tempo_comparison_report_df

,SongName,LabelledTempo,ComputedTempo (Musa),ComputedTempo (Librosa),TempoError (Musa),TempoError (Librosa)
0,Dhanda Nyoliwala - Russian Bandana.mp3,83,166,83,0.0,0.00
1,"Badshah, Karan Aujla, Hiten - God Damn.mp3",95,187,93,1.5,2.00
2,"Atif Aslam, Shreya Ghoshal, Sachin-Jigar - Piy...",84,166,166,1.0,1.00
3,"Vishal Mishra, Rashmeet Kaur - Janiye (from th...",76,149,151,1.5,0.50
4,"Jasleen Royal, Ujjwal Kashyap - Sang Rahiyo.mp3",95,187,93,1.5,2.00
5,"Bayaan, Hasan Raheem, Rovalio - Maand.mp3",140,139,139,1.0,1.00
6,"Atif Aslam, Alisha Chinai, Pritam - Tera Hone ...",120,120,120,0.0,0.00
7,"Cheema Y, Gur Sidhu, Jasmeen Akhtar - Police.mp3",90,181,90,0.5,0.00
8,"Faheem Abdullah, Rauhan Malik, Amir Ameer - Is...",95,193,95,1.5,0.00
9,Diljit Dosanjh - Born to Shine.mp3,90,181,90,0.5,0.00


In [6]:
tempo_comparison_report_summary

,TotalTempoError (Musa),TotalTempoError (Librosa),AvgTempoError (Musa),AvgTempoError (Librosa)
0,79.5,108.75,3.18,4.35


After this analysis, I felt the need to monitor `librosa` tempo detection closely. Let us do an experiment.

In [7]:
# Importing stuff, as usual
import librosa
import numpy as np
import IPython.display as ipd

from musa.music import MusicLoader
from musa.tempo import TempoDetector
from musa.plot import Plotter

music_loader = MusicLoader()
tempo_detector = TempoDetector()
plotter = Plotter()

In [8]:
SR = 44100

def generate_metronome(bpm: int, duration_min: int):

    beat_dur_sec = (60 / bpm) # Seconds
    duration_sec = duration_min * 60 # Seconds
    beat_pos_sec = np.arange(0, duration_sec, beat_dur_sec)
    metronome = librosa.clicks(times=beat_pos_sec, sr=SR, click_freq=1000, click_duration=0.05)

    return metronome

metronomes = [] # List storing metronomes at different BPMs

for bpm in range(80, 300, 10):
    metro = generate_metronome(bpm=bpm, duration_min=1)
    music = music_loader.load_from_signal(x=metro, sr=SR, name=f"metro-{bpm}")
    metronomes.append(music)

In [9]:
tempos_musa = []
tempos_librosa = []

for metronome in metronomes:
    tempo_musa = tempo_detector.detect_tempo(music=metronome)
    tempo_librosa = tempo_detector.detect_tempo_with_librosa(music=metronome)
    
    tempos_musa.append(tempo_musa.bpm)
    tempos_librosa.append(tempo_librosa.bpm)

    print(f"========= {metronome.name} ==============")
    print(f"Librosa: {tempo_librosa.bpm}  |  Custom: {tempo_musa.bpm}\n\n")

========= metro-80 ==============
Librosa: 79  |  Custom: 80


========= metro-90 ==============
Librosa: 90  |  Custom: 89


========= metro-100 ==============
Librosa: 99  |  Custom: 99


========= metro-110 ==============
Librosa: 109  |  Custom: 111


========= metro-120 ==============
Librosa: 120  |  Custom: 120


========= metro-130 ==============
Librosa: 129  |  Custom: 130


========= metro-140 ==============
Librosa: 139  |  Custom: 139


========= metro-150 ==============
Librosa: 74  |  Custom: 150


========= metro-160 ==============
Librosa: 161  |  Custom: 162


========= metro-170 ==============
Librosa: 84  |  Custom: 171


========= metro-180 ==============
Librosa: 178  |  Custom: 181


========= metro-190 ==============
Librosa: 191  |  Custom: 187


========= metro-200 ==============
Librosa: 99  |  Custom: 200


========= metro-210 ==============
Librosa: 105  |  Custom: 127


========= metro-220 ==============
Librosa: 109  |  Custom: 107


========= metro-230 =